# ASL Recognition with TGCN - Complete Pipeline

## 🚀 Improved TGCN with Face Landmarks (553 Nodes)

This notebook implements a state-of-the-art ASL recognition system using:

- **553 keypoints**: 33 pose + 42 hands + 478 face landmarks
- **Advanced preprocessing**: Spatial anchoring, temporal smoothing, interpolation
- **Improved graph connectivity**: Anatomical + functional relationships
- **Data augmentation**: Spatial and temporal transformations
- **WLASL-100 subset**: Focus on quality over quantity

### Architecture Overview

- **Input**: MediaPipe keypoint sequences (seq_len, 553, 3)
- **Graph**: Enhanced connectivity with face-hand relationships
- **Model**: ST-GCN with temporal convolutions
- **Target**: 87.60% accuracy on WLASL-100 (literature benchmark)


In [5]:
# Core libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR

# PyTorch Geometric for GCN
try:
    import torch_geometric
    from torch_geometric.nn import GCNConv, global_mean_pool
    from torch_geometric.data import Data, Batch
    print(f"✅ PyTorch Geometric {torch_geometric.__version__} loaded")
except ImportError:
    print("❌ PyTorch Geometric not found. Install with: pip install torch-geometric")
    raise

# Data handling and visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from collections import defaultdict
import glob
import os
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Import our improved normalization module
from normalization import (
    ImprovedPoseNormalizer,
    create_improved_pose_dataset_class,
    create_improved_graph_connectivity,
    apply_spatial_augmentation,
    apply_temporal_augmentation
)

print("🎯 All libraries loaded successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

✅ PyTorch Geometric 2.6.1 loaded
🚀 Improved normalization and preprocessing module loaded!
📚 Based on successful TGCN implementations achieving 87.60% on WLASL-100
✨ Features: spatial anchoring, temporal smoothing, improved graph connectivity
📊 Supporting 553-node architecture: 33 pose + 42 hands + 478 face landmarks
🎯 All libraries loaded successfully!
PyTorch version: 2.7.0+cu118
CUDA available: True
Using device: cuda
🚀 Improved normalization and preprocessing module loaded!
📚 Based on successful TGCN implementations achieving 87.60% on WLASL-100
✨ Features: spatial anchoring, temporal smoothing, improved graph connectivity
📊 Supporting 553-node architecture: 33 pose + 42 hands + 478 face landmarks
🎯 All libraries loaded successfully!
PyTorch version: 2.7.0+cu118
CUDA available: True
Using device: cuda


## 📊 Configuration and Data Paths

Set up all paths and hyperparameters for the training pipeline.


In [6]:
# Data paths
DATA_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints'
CHECKPOINT_DIR = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints'
MODEL_SAVE_PATH = os.path.join(CHECKPOINT_DIR, 'best_tgcn_face_model.pth')

# Create directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Model hyperparameters
CONFIG = {
    # Data parameters
    'max_seq_len': 50,          # Maximum sequence length
    'num_nodes': 553,           # 33 pose + 42 hands + 478 face
    'num_features': 3,          # x, y, z coordinates
    'max_classes': 100,         # Use WLASL-100 subset
    'test_size': 0.2,           # Train/test split ratio
    'batch_size': 16,           # Batch size (reduced for 553 nodes)
    
    # Model architecture
    'gcn_hidden': 256,          # GCN hidden dimensions
    'temporal_kernel': 9,       # Temporal convolution kernel size
    'dropout': 0.3,             # Dropout rate
    'num_gcn_layers': 3,        # Number of GCN layers
    
    # Training parameters
    'num_epochs': 100,          # Maximum epochs
    'learning_rate': 0.001,     # Initial learning rate
    'weight_decay': 1e-4,       # L2 regularization
    'patience': 15,             # Early stopping patience
    'min_lr': 1e-6,             # Minimum learning rate
    
    # Data augmentation
    'use_augmentation': True,   # Enable data augmentation
    'aug_probability': 0.3,     # Probability of applying augmentation
    'spatial_aug_strength': 0.1, # Spatial augmentation strength
    'temporal_aug_strength': 0.2, # Temporal augmentation strength
}

print("📋 Configuration:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

print(f"\n📁 Data directory: {DATA_DIR}")
print(f"💾 Checkpoint directory: {CHECKPOINT_DIR}")

📋 Configuration:
  max_seq_len: 50
  num_nodes: 553
  num_features: 3
  max_classes: 100
  test_size: 0.2
  batch_size: 16
  gcn_hidden: 256
  temporal_kernel: 9
  dropout: 0.3
  num_gcn_layers: 3
  num_epochs: 100
  learning_rate: 0.001
  weight_decay: 0.0001
  patience: 15
  min_lr: 1e-06
  use_augmentation: True
  aug_probability: 0.3
  spatial_aug_strength: 0.1
  temporal_aug_strength: 0.2

📁 Data directory: f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\data\keypoints
💾 Checkpoint directory: f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\checkpoints


## 🔍 Data Exploration and Validation

Explore the keypoint data to understand the dataset structure and validate the 553-node architecture.


In [ ]:
def explore_dataset(data_dir):
    """Explore the keypoint dataset structure and statistics"""
    
    if not os.path.exists(data_dir):
        print(f"❌ Data directory not found: {data_dir}")
        print("Please run the keypoint extraction first with pose_estimation_mediapipe.py")
        return None
    
    # Find all word directories
    word_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    word_dirs = sorted(word_dirs)
    
    print(f"📊 Dataset Statistics:")
    print(f"  Total classes found: {len(word_dirs)}")
    
    # Analyze sample distribution
    class_stats = []
    total_files = 0
    sample_shapes = []
    
    for word in word_dirs[:20]:  # Check first 20 classes
        word_dir = os.path.join(data_dir, word)
        npz_files = glob.glob(os.path.join(word_dir, "*.npz"))
        total_files += len(npz_files)
        
        # Check sample file shape
        if npz_files:
            try:
                sample_data = np.load(npz_files[0])
                if 'nodes' in sample_data:
                    shape = sample_data['nodes'].shape
                    sample_shapes.append(shape)
                    print(f"  {word}: {len(npz_files)} files, shape: {shape}")
            except Exception as e:
                print(f"  {word}: {len(npz_files)} files, error reading: {e}")
        
        class_stats.append((word, len(npz_files)))
    
    print(f"\n📈 Sample distribution (top 20):")
    class_stats.sort(key=lambda x: x[1], reverse=True)
    for word, count in class_stats[:10]:
        print(f"  {word}: {count} samples")
    
    # Validate node architecture
    if sample_shapes:
        most_common_shape = max(set(sample_shapes), key=sample_shapes.count)
        print(f"\n🏗️ Architecture validation:")
        print(f"  Most common shape: {most_common_shape}")
        print(f"  Expected nodes: {CONFIG['num_nodes']} (33 pose + 42 hands + 478 face)")
        
        if most_common_shape[1] == CONFIG['num_nodes']:
            print(f"  ✅ Architecture matches! Found {most_common_shape[1]} nodes")
        else:
            print(f"  ⚠️ Architecture mismatch! Found {most_common_shape[1]}, expected {CONFIG['num_nodes']}")
            if most_common_shape[1] == 75:
                print(f"  📝 Data contains only pose+hands (75 nodes). Need to re-run extraction with face landmarks.")
                return False
    
    return True

# Explore the dataset
data_ready = explore_dataset(DATA_DIR)

## 🗃️ Dataset Class and Data Loading

Create the dataset class with improved normalization and load the data for training.


In [ ]:
# Create the improved dataset class
ImprovedPoseSequenceDataset = create_improved_pose_dataset_class()

# Initialize datasets
if data_ready:
    print("🔄 Creating datasets...")
    
    # Training dataset
    train_dataset = ImprovedPoseSequenceDataset(
        data_dir=DATA_DIR,
        max_seq_len=CONFIG['max_seq_len'],
        split='train',
        test_size=CONFIG['test_size'],
        random_state=42,
        use_subset=True,
        max_classes=CONFIG['max_classes']
    )
    
    # Test dataset
    test_dataset = ImprovedPoseSequenceDataset(
        data_dir=DATA_DIR,
        max_seq_len=CONFIG['max_seq_len'], 
        split='test',
        test_size=CONFIG['test_size'],
        random_state=42,
        use_subset=True,
        max_classes=CONFIG['max_classes']
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=True,
        num_workers=4,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=CONFIG['batch_size'],
        shuffle=False,
        num_workers=4,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    print(f"✅ Datasets created successfully!")
    print(f"📊 Training samples: {len(train_dataset)}")
    print(f"📊 Test samples: {len(test_dataset)}")
    print(f"📊 Number of classes: {train_dataset.num_classes}")
    print(f"📊 Batch size: {CONFIG['batch_size']}")
    
    # Save class mapping
    class_mapping = {
        'word_to_idx': train_dataset.word_to_idx,
        'idx_to_word': train_dataset.idx_to_word
    }
    
    with open(os.path.join(CHECKPOINT_DIR, 'class_mapping.json'), 'w') as f:
        json.dump(class_mapping, f, indent=2)
    
    print(f"💾 Class mapping saved to {CHECKPOINT_DIR}/class_mapping.json")
    
else:
    print("❌ Cannot create datasets. Please fix data issues first.")

## 🏗️ TGCN Model Architecture

Implement the Temporal Graph Convolutional Network with improved connectivity for 553 nodes.


In [ ]:
class TemporalGCN(nn.Module):
    """Temporal Graph Convolutional Network for ASL Recognition"""
    
    def __init__(self, num_nodes, num_features, num_classes, 
                 gcn_hidden=256, temporal_kernel=9, dropout=0.3, num_gcn_layers=3):
        super(TemporalGCN, self).__init__()
        
        self.num_nodes = num_nodes
        self.num_features = num_features
        self.num_classes = num_classes
        self.gcn_hidden = gcn_hidden
        self.temporal_kernel = temporal_kernel
        
        # Create improved graph connectivity
        self.edge_index = create_improved_graph_connectivity()
        
        # Input projection
        self.input_projection = nn.Linear(num_features, gcn_hidden)
        
        # GCN layers with residual connections
        self.gcn_layers = nn.ModuleList()
        for i in range(num_gcn_layers):
            self.gcn_layers.append(GCNConv(gcn_hidden, gcn_hidden))
        
        # Batch normalization for each GCN layer
        self.batch_norms = nn.ModuleList([
            nn.BatchNorm1d(gcn_hidden) for _ in range(num_gcn_layers)
        ])
        
        # Temporal convolution layers
        self.temporal_conv1 = nn.Conv1d(
            gcn_hidden, gcn_hidden, 
            kernel_size=temporal_kernel,
            padding=temporal_kernel//2
        )
        self.temporal_conv2 = nn.Conv1d(
            gcn_hidden, gcn_hidden//2,
            kernel_size=temporal_kernel,
            padding=temporal_kernel//2
        )
        
        # Dropout layers
        self.dropout = nn.Dropout(dropout)
        self.spatial_dropout = nn.Dropout2d(dropout * 0.5)
        
        # Global pooling and classification
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(gcn_hidden//2, gcn_hidden//4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(gcn_hidden//4, num_classes)
        )
        
        # Initialize weights
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize model weights"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        """
        Forward pass
        
        Args:
            x: Input tensor [batch_size, seq_len, num_nodes, num_features]
            
        Returns:
            logits: Class predictions [batch_size, num_classes]
        """
        batch_size, seq_len, num_nodes, num_features = x.shape
        
        # Move edge index to same device as input
        edge_index = self.edge_index.to(x.device)
        
        # Process each frame separately
        frame_outputs = []
        
        for t in range(seq_len):
            # Current frame: [batch_size, num_nodes, num_features]
            frame = x[:, t, :, :]
            
            # Project input features
            h = self.input_projection(frame)  # [batch_size, num_nodes, gcn_hidden]
            
            # Apply GCN layers with residual connections
            for i, (gcn, bn) in enumerate(zip(self.gcn_layers, self.batch_norms)):
                residual = h if i > 0 else None
                
                # Reshape for GCN: [batch_size * num_nodes, gcn_hidden]
                h_flat = h.view(-1, h.size(-1))
                
                # Expand edge index for batch
                batch_edge_index = edge_index.unsqueeze(0).repeat(batch_size, 1, 1)
                batch_edge_index = batch_edge_index + torch.arange(batch_size, device=x.device).view(-1, 1, 1) * num_nodes
                batch_edge_index = batch_edge_index.view(2, -1)
                
                # Apply GCN
                h_flat = gcn(h_flat, batch_edge_index)
                h = h_flat.view(batch_size, num_nodes, -1)
                
                # Batch normalization
                h = h.permute(0, 2, 1)  # [batch_size, gcn_hidden, num_nodes]
                h = bn(h)
                h = h.permute(0, 2, 1)  # [batch_size, num_nodes, gcn_hidden]
                
                # Activation and residual connection
                h = F.relu(h)
                if residual is not None:
                    h = h + residual
                
                h = self.dropout(h)
            
            # Spatial dropout for regularization
            h = h.unsqueeze(-1)  # [batch_size, num_nodes, gcn_hidden, 1]
            h = self.spatial_dropout(h)
            h = h.squeeze(-1)   # [batch_size, num_nodes, gcn_hidden]
            
            # Global spatial pooling for this frame
            frame_features = torch.mean(h, dim=1)  # [batch_size, gcn_hidden]
            frame_outputs.append(frame_features)
        
        # Stack frame features: [batch_size, seq_len, gcn_hidden]
        temporal_features = torch.stack(frame_outputs, dim=1)
        
        # Temporal convolution: [batch_size, gcn_hidden, seq_len]
        temporal_features = temporal_features.permute(0, 2, 1)
        
        # Apply temporal convolutions
        temporal_features = F.relu(self.temporal_conv1(temporal_features))
        temporal_features = self.dropout(temporal_features)
        temporal_features = F.relu(self.temporal_conv2(temporal_features))
        
        # Global temporal pooling
        sequence_features = self.global_pool(temporal_features).squeeze(-1)  # [batch_size, gcn_hidden//2]
        
        # Classification
        logits = self.classifier(sequence_features)
        
        return logits

print("✅ TGCN model defined successfully!")

## 🛠️ Training Utilities and Metrics

Define training utilities, metrics calculation, and progress tracking functions.


In [ ]:
class MetricsTracker:
    """Track training metrics and progress"""
    
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.train_losses = []
        self.train_accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        self.learning_rates = []
        self.best_val_acc = 0.0
        self.best_epoch = 0
    
    def update(self, train_loss, train_acc, val_loss, val_acc, lr):
        self.train_losses.append(train_loss)
        self.train_accuracies.append(train_acc)
        self.val_losses.append(val_loss)
        self.val_accuracies.append(val_acc)
        self.learning_rates.append(lr)
        
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.best_epoch = len(self.val_accuracies) - 1
    
    def plot_metrics(self):
        """Plot training metrics"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss plot
        axes[0, 0].plot(self.train_losses, label='Train Loss', color='blue')
        axes[0, 0].plot(self.val_losses, label='Val Loss', color='red')
        axes[0, 0].set_title('Training and Validation Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # Accuracy plot
        axes[0, 1].plot(self.train_accuracies, label='Train Acc', color='blue')
        axes[0, 1].plot(self.val_accuracies, label='Val Acc', color='red')
        axes[0, 1].axhline(y=self.best_val_acc, color='green', linestyle='--', 
                          label=f'Best Val Acc: {self.best_val_acc:.3f}')
        axes[0, 1].set_title('Training and Validation Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        axes[0, 1].grid(True)
        
        # Learning rate plot
        axes[1, 0].plot(self.learning_rates, color='orange')
        axes[1, 0].set_title('Learning Rate Schedule')
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Learning Rate')
        axes[1, 0].set_yscale('log')
        axes[1, 0].grid(True)
        
        # Validation accuracy zoomed
        axes[1, 1].plot(self.val_accuracies, color='red', linewidth=2)
        axes[1, 1].axhline(y=self.best_val_acc, color='green', linestyle='--')
        axes[1, 1].set_title(f'Validation Accuracy (Best: {self.best_val_acc:.3f}% at epoch {self.best_epoch})')
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Accuracy (%)')
        axes[1, 1].grid(True)
        
        plt.tight_layout()
        plt.show()

def calculate_accuracy(outputs, targets):
    """Calculate accuracy from model outputs and targets"""
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == targets).sum().item()
    total = targets.size(0)
    return 100.0 * correct / total

def train_epoch(model, train_loader, criterion, optimizer, device, use_augmentation=False):
    """Train for one epoch"""
    model.train()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)
        
        # Apply data augmentation if enabled
        if use_augmentation and np.random.random() < CONFIG['aug_probability']:
            # Convert to numpy for augmentation
            data_np = data.cpu().numpy()
            
            # Apply spatial augmentation
            if np.random.random() < 0.5:
                data_np = apply_spatial_augmentation(
                    data_np, 
                    scale_range=CONFIG['spatial_aug_strength'],
                    translation_range=CONFIG['spatial_aug_strength']
                )
            
            # Apply temporal augmentation  
            if np.random.random() < 0.5:
                augmented_batch = []
                for i in range(data_np.shape[0]):
                    aug_seq = apply_temporal_augmentation(
                        data_np[i], 
                        speed_range=CONFIG['temporal_aug_strength']
                    )
                    # Ensure consistent length
                    if aug_seq.shape[0] != CONFIG['max_seq_len']:
                        if aug_seq.shape[0] > CONFIG['max_seq_len']:
                            indices = np.linspace(0, aug_seq.shape[0]-1, CONFIG['max_seq_len'], dtype=int)
                            aug_seq = aug_seq[indices]
                        else:
                            padding = np.zeros((CONFIG['max_seq_len'] - aug_seq.shape[0], 
                                              aug_seq.shape[1], aug_seq.shape[2]))
                            aug_seq = np.concatenate([aug_seq, padding], axis=0)
                    augmented_batch.append(aug_seq)
                
                data = torch.tensor(np.array(augmented_batch), dtype=torch.float32).to(device)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
        total_accuracy += calculate_accuracy(outputs, targets)
        num_batches += 1
        
        if batch_idx % 20 == 0:
            print(f'  Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}')
    
    return total_loss / num_batches, total_accuracy / num_batches

def validate_epoch(model, val_loader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0
    
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            total_loss += loss.item()
            total_accuracy += calculate_accuracy(outputs, targets)
            num_batches += 1
    
    return total_loss / num_batches, total_accuracy / num_batches

print("✅ Training utilities defined successfully!")

## 🚀 Model Initialization and Training Setup

Initialize the TGCN model and set up the training components.


In [ ]:
if data_ready:
    # Initialize model
    model = TemporalGCN(
        num_nodes=CONFIG['num_nodes'],
        num_features=CONFIG['num_features'],
        num_classes=train_dataset.num_classes,
        gcn_hidden=CONFIG['gcn_hidden'],
        temporal_kernel=CONFIG['temporal_kernel'],
        dropout=CONFIG['dropout'],
        num_gcn_layers=CONFIG['num_gcn_layers']
    ).to(device)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"🏗️ Model Architecture:")
    print(f"  Total parameters: {total_params:,}")
    print(f"  Trainable parameters: {trainable_params:,}")
    print(f"  Model size: {total_params * 4 / 1024 / 1024:.2f} MB")
    
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=CONFIG['learning_rate'],
        weight_decay=CONFIG['weight_decay']
    )
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='max',
        factor=0.5,
        patience=5,
        min_lr=CONFIG['min_lr'],
        verbose=True
    )
    
    # Metrics tracker
    metrics = MetricsTracker()
    
    print(f"✅ Training setup complete!")
    print(f"  Device: {device}")
    print(f"  Optimizer: AdamW")
    print(f"  Scheduler: ReduceLROnPlateau")
    print(f"  Loss function: CrossEntropyLoss")
    
    # Test a forward pass
    model.eval()
    with torch.no_grad():
        test_batch = next(iter(train_loader))
        test_data, test_targets = test_batch[0][:2].to(device), test_batch[1][:2].to(device)
        test_output = model(test_data)
        print(f"\n🧪 Test forward pass:")
        print(f"  Input shape: {test_data.shape}")
        print(f"  Output shape: {test_output.shape}")
        print(f"  Expected output shape: [2, {train_dataset.num_classes}]")
        
        if test_output.shape == (2, train_dataset.num_classes):
            print(f"  ✅ Forward pass successful!")
        else:
            print(f"  ❌ Shape mismatch in forward pass!")

else:
    print("❌ Cannot initialize model. Please fix data issues first.")

## 🎯 Training Loop

Execute the main training loop with validation, early stopping, and progress tracking.


In [ ]:
if data_ready:
    print("🚀 Starting training...")
    print(f"Target: 87.60% accuracy on WLASL-100 (literature benchmark)\n")
    
    best_val_acc = 0.0
    patience_counter = 0
    
    for epoch in range(CONFIG['num_epochs']):
        print(f"\n{'='*60}")
        print(f"Epoch {epoch+1}/{CONFIG['num_epochs']}")
        print(f"Learning Rate: {optimizer.param_groups[0]['lr']:.2e}")
        print(f"{'='*60}")
        
        # Training phase
        print("🏋️ Training...")
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, device, 
            use_augmentation=CONFIG['use_augmentation']
        )
        
        # Validation phase
        print("\n🔬 Validating...")
        val_loss, val_acc = validate_epoch(model, test_loader, criterion, device)
        
        # Update metrics
        current_lr = optimizer.param_groups[0]['lr']
        metrics.update(train_loss, train_acc, val_loss, val_acc, current_lr)
        
        # Learning rate scheduling
        scheduler.step(val_acc)
        
        # Print epoch results
        print(f"\n📊 Epoch {epoch+1} Results:")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"  Best Val Acc: {metrics.best_val_acc:.2f}% (Epoch {metrics.best_epoch+1})")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_acc': best_val_acc,
                'config': CONFIG,
                'class_mapping': class_mapping
            }, MODEL_SAVE_PATH)
            
            print(f"  💾 New best model saved! Val Acc: {val_acc:.2f}%")
            
            # Check if we've reached the target accuracy
            if val_acc >= 87.60:
                print(f"\n🎉 TARGET ACHIEVED! Validation accuracy: {val_acc:.2f}% >= 87.60%")
                print(f"Training completed successfully at epoch {epoch+1}")
                break
        
        else:
            patience_counter += 1
            print(f"  ⏳ Patience: {patience_counter}/{CONFIG['patience']}")
            
            # Early stopping
            if patience_counter >= CONFIG['patience']:
                print(f"\n⛔ Early stopping triggered at epoch {epoch+1}")
                print(f"Best validation accuracy: {best_val_acc:.2f}%")
                break
        
        # Stop if learning rate becomes too small
        if current_lr < CONFIG['min_lr']:
            print(f"\n⛔ Learning rate too small: {current_lr:.2e}")
            break
    
    print(f"\n🏁 Training completed!")
    print(f"Best validation accuracy: {metrics.best_val_acc:.2f}%")
    print(f"Model saved to: {MODEL_SAVE_PATH}")
    
    # Plot training metrics
    print("\n📈 Training Metrics:")
    metrics.plot_metrics()

else:
    print("❌ Cannot start training. Please fix data issues first.")

## 📊 Model Evaluation and Analysis

Evaluate the trained model and analyze its performance in detail.


In [ ]:
def evaluate_model(model, test_loader, class_mapping, device):
    """Comprehensive model evaluation"""
    model.eval()
    
    all_predictions = []
    all_targets = []
    all_outputs = []
    
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            
            _, predicted = torch.max(outputs, 1)
            
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_outputs.extend(F.softmax(outputs, dim=1).cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_targets, all_predictions)
    
    # Classification report
    idx_to_word = class_mapping['idx_to_word']
    class_names = [idx_to_word[str(i)] for i in range(len(idx_to_word))]
    
    print(f"\n📊 Final Evaluation Results:")
    print(f"Overall Accuracy: {accuracy*100:.2f}%")
    
    # Detailed classification report
    report = classification_report(
        all_targets, all_predictions, 
        target_names=class_names,
        output_dict=True
    )
    
    print(f"\n📋 Classification Report:")
    print(classification_report(all_targets, all_predictions, target_names=class_names))
    
    # Confusion matrix visualization
    cm = confusion_matrix(all_targets, all_predictions)
    
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names[:20],  # Show first 20 classes
                yticklabels=class_names[:20])
    plt.title('Confusion Matrix (First 20 Classes)')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Per-class accuracy analysis
    class_accuracies = []
    for i in range(len(class_names)):
        class_mask = np.array(all_targets) == i
        if np.sum(class_mask) > 0:
            class_acc = accuracy_score(
                np.array(all_targets)[class_mask],
                np.array(all_predictions)[class_mask]
            )
            class_accuracies.append((class_names[i], class_acc, np.sum(class_mask)))
    
    # Sort by accuracy
    class_accuracies.sort(key=lambda x: x[1], reverse=True)
    
    print(f"\n🎯 Top 10 Best Performing Classes:")
    for name, acc, count in class_accuracies[:10]:
        print(f"  {name}: {acc*100:.1f}% ({count} samples)")
    
    print(f"\n🎯 Bottom 10 Performing Classes:")
    for name, acc, count in class_accuracies[-10:]:
        print(f"  {name}: {acc*100:.1f}% ({count} samples)")
    
    return accuracy, report, cm

if data_ready and os.path.exists(MODEL_SAVE_PATH):
    print("🔄 Loading best model for evaluation...")
    
    # Load the best model
    checkpoint = torch.load(MODEL_SAVE_PATH, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"✅ Model loaded from epoch {checkpoint['epoch']+1}")
    print(f"Best validation accuracy: {checkpoint['best_val_acc']:.2f}%")
    
    # Evaluate on test set
    accuracy, report, cm = evaluate_model(model, test_loader, class_mapping, device)
    
    # Compare with literature
    print(f"\n🏆 Performance Comparison:")
    print(f"Our Model: {accuracy*100:.2f}%")
    print(f"Literature Benchmark (WLASL-100): 87.60%")
    
    if accuracy*100 >= 87.60:
        print(f"🎉 SUCCESS! We've achieved the literature benchmark!")
    elif accuracy*100 >= 80.0:
        print(f"✅ GOOD! Strong performance, close to benchmark")
    elif accuracy*100 >= 70.0:
        print(f"⚠️ MODERATE: Decent performance, room for improvement")
    else:
        print(f"❌ POOR: Significant improvement needed")
    
    # Save evaluation results
    eval_results = {
        'accuracy': accuracy,
        'classification_report': report,
        'model_path': MODEL_SAVE_PATH,
        'config': CONFIG
    }
    
    with open(os.path.join(CHECKPOINT_DIR, 'evaluation_results.json'), 'w') as f:
        json.dump(eval_results, f, indent=2, default=str)
    
    print(f"\n💾 Evaluation results saved to {CHECKPOINT_DIR}/evaluation_results.json")

else:
    if not data_ready:
        print("❌ Cannot evaluate: Data not ready")
    else:
        print(f"❌ Cannot evaluate: Model not found at {MODEL_SAVE_PATH}")
        print("Please run the training first.")

## 🔮 Inference and Model Usage

Functions for using the trained model to make predictions on new data.


In [ ]:
def load_trained_model(model_path, device):
    """Load a trained model for inference"""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model not found: {model_path}")
    
    checkpoint = torch.load(model_path, map_location=device)
    config = checkpoint['config']
    class_mapping = checkpoint['class_mapping']
    
    # Initialize model with saved config
    model = TemporalGCN(
        num_nodes=config['num_nodes'],
        num_features=config['num_features'],
        num_classes=len(class_mapping['idx_to_word']),
        gcn_hidden=config['gcn_hidden'],
        temporal_kernel=config['temporal_kernel'],
        dropout=config['dropout'],
        num_gcn_layers=config['num_gcn_layers']
    ).to(device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    return model, class_mapping, config

def predict_sequence(model, keypoints, class_mapping, config, device):
    """
    Predict ASL sign from keypoint sequence
    
    Args:
        model: Trained TGCN model
        keypoints: numpy array [seq_len, num_nodes, 3]
        class_mapping: Dictionary with class mappings
        config: Model configuration
        device: PyTorch device
    
    Returns:
        predicted_class: string
        confidence: float
        all_probabilities: dict
    """
    # Normalize keypoints
    normalizer = ImprovedPoseNormalizer()
    normalized_keypoints = normalizer.normalize_pose_sequence(keypoints)
    
    # Handle sequence length
    seq_len = normalized_keypoints.shape[0]
    if seq_len > config['max_seq_len']:
        # Resample to max length
        indices = np.linspace(0, seq_len - 1, config['max_seq_len'], dtype=int)
        normalized_keypoints = normalized_keypoints[indices]
    elif seq_len < config['max_seq_len']:
        # Pad sequence
        padding = np.zeros((config['max_seq_len'] - seq_len, 
                          normalized_keypoints.shape[1], 
                          normalized_keypoints.shape[2]))
        normalized_keypoints = np.concatenate([normalized_keypoints, padding], axis=0)
    
    # Convert to tensor and add batch dimension
    input_tensor = torch.tensor(normalized_keypoints, dtype=torch.float32)
    input_tensor = input_tensor.unsqueeze(0).to(device)  # [1, seq_len, num_nodes, 3]
    
    # Make prediction
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted_idx = torch.max(probabilities, 1)
    
    # Convert to readable format
    predicted_class = class_mapping['idx_to_word'][str(predicted_idx.item())]
    confidence_score = confidence.item()
    
    # Get all class probabilities
    all_probs = {}
    prob_array = probabilities[0].cpu().numpy()
    for idx, prob in enumerate(prob_array):
        if str(idx) in class_mapping['idx_to_word']:
            class_name = class_mapping['idx_to_word'][str(idx)]
            all_probs[class_name] = float(prob)
    
    return predicted_class, confidence_score, all_probs

def inference_demo():
    """Demonstrate inference on a sample from the test set"""
    if not data_ready or not os.path.exists(MODEL_SAVE_PATH):
        print("❌ Cannot run demo: Model or data not available")
        return
    
    print("🔮 Running inference demo...")
    
    # Load trained model
    model, class_mapping, config = load_trained_model(MODEL_SAVE_PATH, device)
    
    # Get a random sample from test set
    test_sample_idx = np.random.randint(0, len(test_dataset))
    sample_data, true_label = test_dataset[test_sample_idx]
    
    # Convert tensor back to numpy for prediction function
    keypoints_np = sample_data.cpu().numpy()  # [seq_len, num_nodes, 3]
    true_class = class_mapping['idx_to_word'][str(true_label.item())]
    
    # Make prediction
    predicted_class, confidence, all_probs = predict_sequence(
        model, keypoints_np, class_mapping, config, device
    )
    
    print(f"\n🎯 Inference Results:")
    print(f"True class: {true_class}")
    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.3f}")
    print(f"Correct: {'✅' if predicted_class == true_class else '❌'}")
    
    # Show top 5 predictions
    sorted_probs = sorted(all_probs.items(), key=lambda x: x[1], reverse=True)
    print(f"\n📊 Top 5 Predictions:")
    for i, (class_name, prob) in enumerate(sorted_probs[:5]):
        marker = "👑" if class_name == true_class else "  "
        print(f"  {marker} {i+1}. {class_name}: {prob:.3f}")

# Run inference demo
if data_ready:
    inference_demo()

print("\n✅ Inference functions defined successfully!")
print("💡 Use predict_sequence() function to make predictions on new keypoint data")

## 📋 Summary and Next Steps

### 🎉 What We've Accomplished

1. **Enhanced Architecture**: 553-node TGCN with pose, hands, and face landmarks
2. **Advanced Preprocessing**: Spatial anchoring, temporal smoothing, interpolation
3. **Improved Connectivity**: Anatomical and functional graph relationships
4. **Data Augmentation**: Spatial and temporal transformations
5. **Robust Training**: Early stopping, learning rate scheduling, gradient clipping
6. **Comprehensive Evaluation**: Metrics, visualizations, and analysis

### 🎯 Performance Target

- **Goal**: 87.60% accuracy on WLASL-100 (literature benchmark)
- **Architecture**: 553 nodes (33 pose + 42 hands + 478 face)
- **Dataset**: WLASL-100 subset for quality training

### 🚀 Next Steps

1. **Data Extraction**: Run `pose_estimation_mediapipe.py` to extract 553-node keypoints
2. **Training**: Execute this notebook to train the TGCN model
3. **Evaluation**: Analyze performance and compare with literature
4. **Optimization**: Tune hyperparameters if needed
5. **Production**: Use inference functions for real-time ASL recognition

### 📁 File Structure

```
pose_estimation/src/
├── tgcn_pipeline.ipynb     # This comprehensive notebook
├── normalization.py        # Advanced preprocessing utilities
└── preprocessing/
    └── pose_estimation_mediapipe.py  # 553-node keypoint extraction
```

### 🔧 Key Configuration

- **Nodes**: 553 (pose + hands + face)
- **Sequence Length**: 50 frames
- **Batch Size**: 16 (optimized for 553 nodes)
- **Classes**: 100 (WLASL-100 subset)
- **Augmentation**: Spatial + temporal

---

**Ready to achieve state-of-the-art ASL recognition performance! 🚀**


## 🧪 Quick Test: Verify 553-Node Keypoint Format

Before training, let's verify that our keypoint extraction produced the correct 553-node format.


In [7]:
from pathlib import Path

def test_keypoint_format_quick():
    """Quick test to verify 553-node keypoint format"""
    print("🔍 Testing 553-node keypoint extraction format...")
    
    keypoints_dir = Path(DATA_DIR)
    keypoint_files = list(keypoints_dir.glob("*/*_keypoints.npz"))
    
    if not keypoint_files:
        print("❌ No keypoint files found. Run pose_estimation_mediapipe.py first!")
        return False
    
    # Test first few files
    test_files = keypoint_files[:3]
    print(f"📁 Testing {len(test_files)} sample files...")
    
    for i, file_path in enumerate(test_files):
        print(f"\n{i+1}. {file_path.parent.name}/{file_path.name}")
        
        try:
            data = np.load(str(file_path))
            
            if 'nodes' in data:
                nodes = data['nodes']
                frames, num_nodes, coords = nodes.shape
                
                if num_nodes == 553 and coords == 3:
                    print(f"   ✅ Correct format: {frames} frames × 553 nodes × 3 coords")
                    
                    # Quick detection check
                    pose_detected = np.any(nodes[0, 0:33, :] != 0)
                    hands_detected = np.any(nodes[0, 33:75, :] != 0)
                    face_detected = np.any(nodes[0, 75:553, :] != 0)
                    
                    print(f"   🎯 Detection: Pose {'✅' if pose_detected else '❌'} | "
                          f"Hands {'✅' if hands_detected else '❌'} | "
                          f"Face {'✅' if face_detected else '❌'}")
                    
                elif num_nodes == 75:
                    print(f"   ⚠️  Old format detected: {frames} frames × 75 nodes × 3 coords")
                    print(f"   💡 Re-run pose_estimation_mediapipe.py for 553-node extraction")
                    return False
                else:
                    print(f"   ❌ Unexpected format: {frames} frames × {num_nodes} nodes × {coords} coords")
                    return False
            else:
                print(f"   ❌ No 'nodes' key found")
                return False
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
            return False
    
    print(f"\n🎉 SUCCESS: All tested files have correct 553-node format!")
    print(f"📊 Total keypoint files available: {len(keypoint_files)}")
    return True

# Run the test
format_test_passed = test_keypoint_format_quick()

if format_test_passed:
    print("\n✅ Ready to proceed with training!")
else:
    print("\n⚠️  Please fix keypoint format before training.")

🔍 Testing 553-node keypoint extraction format...
📁 Testing 3 sample files...

1. about/00414_keypoints.npz
   ✅ Correct format: 104 frames × 553 nodes × 3 coords
   🎯 Detection: Pose ✅ | Hands ❌ | Face ✅

2. about/00415_keypoints.npz
   ✅ Correct format: 37 frames × 553 nodes × 3 coords
   🎯 Detection: Pose ✅ | Hands ❌ | Face ✅

3. about/00416_keypoints.npz
   ✅ Correct format: 115 frames × 553 nodes × 3 coords
   🎯 Detection: Pose ✅ | Hands ❌ | Face ✅

🎉 SUCCESS: All tested files have correct 553-node format!
📊 Total keypoint files available: 203

✅ Ready to proceed with training!
   ✅ Correct format: 37 frames × 553 nodes × 3 coords
   🎯 Detection: Pose ✅ | Hands ❌ | Face ✅

3. about/00416_keypoints.npz
   ✅ Correct format: 115 frames × 553 nodes × 3 coords
   🎯 Detection: Pose ✅ | Hands ❌ | Face ✅

🎉 SUCCESS: All tested files have correct 553-node format!
📊 Total keypoint files available: 203

✅ Ready to proceed with training!


## 🚨 CRITICAL: Debug Hand Detection First

Before running the full keypoint extraction, let's debug hand detection to ensure ASL hands are properly detected. This is CRITICAL for ASL recognition success.


In [ ]:
# CRITICAL: Run hand detection debugging first!
# This will test MediaPipe hand detection on sample ASL videos

print("🚨 CRITICAL: Testing hand detection before full extraction")
print("This step is essential for ASL recognition success!")
print("")

# Check if debug script exists
import os
debug_script = r'f:\Uni_Stuff\6th_Sem\DL\Proj\video-asl-recognition\pose_estimation\src\preprocessing\debug_hand_detection.py'

if os.path.exists(debug_script):
    print("✅ Debug script found. Run this in terminal:")
    print("")
    print("cd \"f:\\Uni_Stuff\\6th_Sem\\DL\\Proj\\video-asl-recognition\\pose_estimation\\src\\preprocessing\"")
    print("python debug_hand_detection.py")
    print("")
    print("This will:")
    print("1. 🔍 Test hand detection on sample ASL videos")
    print("2. 📊 Show detection rates for hands, pose, and face")
    print("3. 💾 Create debug visualizations")
    print("4. 💡 Provide recommendations for optimization")
    print("")
    print("❗ IMPORTANT: Only proceed with full extraction if hand detection >30%")
    print("❗ If hand detection is poor, we'll need to fix MediaPipe settings first")
else:
    print("❌ Debug script not found. Creating it now...")
    # The script was already created above